<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-3h1sg0b_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-3h1sg0b_
  Resolved https://github.com/huggingface/transformers to commit 5330b83bc5637b8e7eafe095c22ef19e21baff2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [17]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [18]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [19]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

In [20]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("medium")

In [21]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:18<00:00, 84.9MiB/s]


In [22]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [23]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr")
    return result["text"]

In [24]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [25]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [26]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.14285714285714285, Precision: 0.8571428571428571, Recall: 0.84, F1: 0.8484848484848485
Processed 2 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 3 files, WER: 0.3793103448275862, Precision: 0.5833333333333334, Recall: 0.5833333333333334, F1: 0.5833333333333334
Processed 4 files, WER: 0.5142857142857142, Precision: 0.46875, Recall: 0.5172413793103449, F1: 0.4918032786885246
Processed 5 files, WER: 0.3888888888888889, Precision: 0.6111111111111112, Recall: 0.6111111111111112, F1: 0.6111111111111112
Processed 6 files, WER: 0.5434782608695652, Precision: 0.46153846153846156, Recall: 0.43902439024390244, F1: 0.45
Processed 7 files, WER: 0.3684210526315789, Precision: 0.6842105263157895, Recall: 0.7222222222222222, F1: 0.7027027027027027
Processed 8 files, WER: 0.575, Precision: 0.5161290322580645, Recall: 0.42105263157894735, F1: 0.46376811594202894
Processed 9 files, WER: 0.3333333333333333, Precision: 0.6451612903225806, Recall: 0.6896551724137931